<a href="https://colab.research.google.com/github/goga0001/graph/blob/main/minimal%20repro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Manual Steps**

---




> Set the runtime to GPU via "Runtime > Change runtime type..."







# **Clone Git repository**

In [1]:
!git clone https://github.com/DeepGraphLearning/torchdrug

Cloning into 'torchdrug'...
remote: Enumerating objects: 1231, done.
remote: Counting objects: 100% (605/605), done.
remote: Compressing objects: 100% (268/268), done.
remote: Total 1231 (delta 366), reused 488 (delta 322), pack-reused 626
Receiving objects: 100% (1231/1231), 2.63 MiB | 9.03 MiB/s, done.
Resolving deltas: 100% (630/630), done.


In [2]:
cd torchdrug

/content/torchdrug


# **Install requirements**

In [3]:
!pip install decorator  rdkit rdkit-pypi torch_geometric==2.0.4 matplotlib  tqdm  networkx   jinja2  lmdb  fair-esm 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 29.1 MB 1.5 MB/s 
     |████████████████████████████████| 29.1 MB 98 kB/s 
     |████████████████████████████████| 407 kB 59.9 MB/s 
     |████████████████████████████████| 93 kB 809 kB/s 
  Created wheel for torch-geometric: filename=torch_geometric-2.0.4-py3-none-any.whl size=616602 sha256=c504886a7838a27de8c601435ed6bdf6044dfc465f3a898ff6d94256e9d314d0
  Stored in directory: /root/.cache/pip/wheels/18/a6/a4/ca18c3051fcead866fe7b85700ee2240d883562a1bc70ce421
Successfully built torch-geometric


In [4]:
!pip install --no-index torch-scatter -f https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
!pip install --no-index torch-sparse -f https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
!pip install --no-index torch-cluster -f https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
!pip install --no-index torch-spline-conv -f https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html

Looking in links: https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
     |████████████████████████████████| 8.9 MB 32.6 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
     |████████████████████████████████| 3.5 MB 30.0 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
     |████████████████████████████████| 2.5 MB 33.0 MB/s 
Looking in links: https://pytorch-geometric.com/whl/torch-1.12.1+cu113.html
     |████████████████████████████████| 722 kB 29.3 MB/s 


In [5]:
pip install fair-esm==0.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: fair-esm
    Found existing installation: fair-esm 2.0.0
    Uninstalling fair-esm-2.0.0:
      Successfully uninstalled fair-esm-2.0.0


In [6]:
!pip install Ninja==1.10.2.4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 120 kB 17.6 MB/s 


In [7]:
pip install torch torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip list

In [8]:
pip -V

pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)


In [9]:
 pip list | grep torch

torch                         1.12.1+cu113
torch-cluster                 1.6.0
torch-geometric               2.0.4
torch-scatter                 2.0.9
torch-sparse                  0.6.15
torch-spline-conv             1.2.1
torchaudio                    0.12.1+cu113
torchsummary                  1.5.1
torchtext                     0.13.1
torchvision                   0.13.1+cu113


# **Run setup.py**

In [141]:
!python setup.py install

python3: can't open file 'setup.py': [Errno 2] No such file or directory


# **Prepare the Pretraining Dataset(5 min)**

 Before registering new dataset- upload csv(https://raw.githubusercontent.com/goga0001/graph/main/data.csv) into dataset folder

In [9]:
import os

from torchdrug import data, utils
from torchdrug.core import Registry as R
from collections import defaultdict

from torch.utils import data as torch_data

from torchdrug import data
from torchdrug.utils import doc


@R.register("datasets.WB")
@utils.copy_args(data.MoleculeDataset.load_csv, ignore=("smiles_field", "target_fields"))
class WB(data.MoleculeDataset):
    """
    Subset of random compound database for virtual screening.

    Statistics:
        - #Molecule:  4806
        - #Regression task: 2

    Parameters:
        path (str): path to store the dataset
        verbose (int, optional): output verbose level
        **kwargs
    """

    csv_file = "/content/torchdrug/torchdrug/datasets/data.csv"
    target_fields = ["logP","qed"]

    def __init__(self, path, verbose=1, **kwargs):
        self.load_csv(self.csv_file, smiles_field="smiles", target_fields=self.target_fields,
                      verbose=verbose, **kwargs)

In [10]:
cd /content/torchdrug/torchdrug/datasets

/content/torchdrug/torchdrug/datasets


In [190]:
import torch
from torchdrug import core, models, tasks


dataset = WB("/content/torchdrug/torchdrug/datasets/data.csv", kekulize=True,
                            atom_feature="symbol" )






Loading /content/torchdrug/torchdrug/datasets/data.csv: 4807it [00:00, 78051.76it/s]            
Constructing molecules from SMILES: 100%|██████████| 4806/4806 [00:08<00:00, 581.06it/s]


# **Define the Model: GraphAF (30sec)**

In [193]:
from torchdrug import core, models, tasks
from torchdrug.layers import distribution

model = models.RGCN(input_dim=dataset.num_atom_type,
                    num_relation=dataset.num_bond_type,
                    hidden_dims=[256, 256, 256], batch_norm=True)

num_atom_type = dataset.num_atom_type
# add one class for non-edge
num_bond_type = dataset.num_bond_type + 1

node_prior = distribution.IndependentGaussian(torch.zeros(num_atom_type),
                                              torch.ones(num_atom_type))
edge_prior = distribution.IndependentGaussian(torch.zeros(num_bond_type),
                                              torch.ones(num_bond_type))
node_flow = models.GraphAF(model, node_prior, num_layer=12)
edge_flow = models.GraphAF(model, edge_prior, use_edge=True, num_layer=12)

task = tasks.AutoregressiveGeneration(node_flow, edge_flow,
                                      max_node=38, max_edge_unroll=12,
                                      criterion="nll")

# **Pretraining and Generation: GraphAF**

In [ ]:
from torch import nn, optim
optimizer = optim.Adam(task.parameters(), lr = 1e-3)
solver = core.Engine(task, dataset, None, None, optimizer,
                     gpus=(0,), batch_size=128, log_interval=10)

solver.train(num_epoch=10)
solver.save("graphaf_WB.pkl")

In [29]:
from collections import defaultdict

solver.load("graphaf_WB.pkl")
results = task.generate(num_sample=32)
print(results.to_smiles())

17:17:52   Load checkpoint from graphaf_flavonoid_1epoch.pkl


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:1949: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


['NCl', 'P', 'Br', 'ClP1(Cl)=P(P(Cl)(Cl)(Cl)P2=P3=S=2(Cl)S3(Cl)(Cl)(Cl)P(Cl)(Cl)(Cl)Cl)=P1', 'Cl', 'O', 'Cl[SH]12(OBr)P=P13PN32', 'P=[PH]1P23=P4(Cl)P5(Cl)=P6=S5(Cl)(Br)S612=P43Cl', 'ClP12(Cl)N=N[PH]1=N2', 'S', 'ClSBr', 'O', 'P', 'CBr', 'Cl', 'CCl', 'S', 'NBr', 'Cl', 'P', 'ClBr', 'PBr', 'ClCl', 'N', 'OCCl', 'N', 'P=S', 'Cl', 'S', 'SCl', 'O', 'C=O']
